In [ ]:
import requests
import pandas as pd
import os

In [ ]:
headers = {"X-eBirdApiToken": API_KEY}

url = "https://api.ebird.org/v2/ref/hotspot/geo"
params = {
    "lat": 45.4215,
    "lng": -75.6972,
    "dist": 10
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()

hotspots = response.json()

df_hotspots = pd.DataFrame(hotspots)
df_hotspots.head()

In [ ]:
url = "https://api.ebird.org/v2/ref/hotspot/geo"
params = {"lat": 40.7128, "lng": -74.0060, "dist": 25}  # NYC
response = requests.get(url, headers=headers, params=params)
print(response.status_code, response.text)

response.body

In [ ]:
import pandas as pd
from io import StringIO

csv_text = response.text

df_hotspots = pd.read_csv(StringIO(csv_text), header=None)

df_hotspots.columns = [
    "locId",
    "countryCode",
    "subnational1Code",
    "subnational2Code",
    "lat",
    "lng",
    "locName",
    "latestObsDt",
    "numSpeciesAllTime"
]

# Quick view
df_hotspots.head()

In [ ]:
df_hotspots

show hotspots on map

In [ ]:
import folium

# Center map on average location
center_lat = df_hotspots["lat"].mean()
center_lng = df_hotspots["lng"].mean()

m = folium.Map(location=[center_lat, center_lng], zoom_start=11)

# Add markers
for _, row in df_hotspots.iterrows():
    popup_text = (
        f"<b>{row['locName']}</b><br>"
        f"Species matched: {row['numSpeciesAllTime']}"
    )
    
    folium.Marker(
        location=[row["lat"], row["lng"]],
        popup=popup_text
    ).add_to(m)

m

* list of desired species
* starting location
* time period
* different color code based on number of species
* ideally made a icon with the picture of the species

In [ ]:
from pathlib import Path
url = "https://api.ebird.org/v2/data/obs/ES/historic/2025/4/1"
response = requests.get(url, headers=headers)
print(response.status_code, response.text)
Path("test.json").write_bytes(response.content)

In [ ]:
Path("test.json").read_text()

In [ ]:
df_day_data = pd.read_json("test.json")

In [ ]:
df_day_data

In [ ]:
df_day_data[["comName"]].drop_duplicates()

In [ ]:
owl_species = [
    "Tawny Owl",
    "Little Owl",
    "Eurasian Scops-Owl",
    "Western Barn Owl",
    "Eurasian Eagle-Owl",
    "Long-eared Owl",
    "Short-eared Owl",
    "Boreal Owl",
    "Eurasian Pygmy Owl",
    "Snowy Owl",
    "Northern Hawk Owl"
]

df_owls = df_day_data[df_day_data["comName"].isin(owl_species)]

In [ ]:
import requests
from pathlib import Path
from datetime import date, timedelta
import time

headers = {"X-eBirdApiToken": API_KEY}

regions = [
    {"code": "ES-CT-BR", "name": "Barcelona"},
    {"code": "ES-CT-GN", "name": "Girona"},
    {"code": "ES-CT-LD", "name": "Lleida"},
    {"code": "ES-CT-TG", "name": "Tarragona"},
]

out_dir = Path("ebird_ES_CT_historic")
out_dir.mkdir(exist_ok=True)

start_year = 2025
end_year = 2025

for region in regions:
    region_code = region["code"]

    base_url = f"https://api.ebird.org/v2/data/obs/{region_code}/historic"

    for year in range(start_year, end_year + 1):
        start_date = date(year, 3, 1)
        end_date = date(year, 3, 31)

        current_date = start_date
        while current_date <= end_date:
            y = current_date.year
            m = current_date.month
            d = current_date.day

            url = f"{base_url}/{y}/{m}/{d}"
            response = requests.get(url, headers=headers)

            print(region_code, y, m, d, response.status_code)

            if response.status_code == 200 and response.text.strip():
                filename = out_dir / f"{region_code}_{y}_{m:02d}_{d:02d}.json"
                filename.write_bytes(response.content)

            # Be polite to the API
            time.sleep(1)

            current_date += timedelta(days=1)


In [ ]:
import json
import pandas as pd
from pathlib import Path

dfs = []

for file in Path("ebird_ES_CT_historic").glob("*.json"):
    data = json.loads(file.read_text())
    if data:
        df = pd.DataFrame(data)
        df["source_file"] = file.name
        dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

In [ ]:
df_all.loc[lambda df:df['comName'] == 'Little Owl'].sort_values(['obsDt'])

In [ ]:
import folium

df_litowl = df_all.loc[lambda df:(df['comName'] == 'Little Owl') & (df['obsDt'].str.contains('2025-03-01'))].sort_values(['obsDt'])

# Center map on average location
center_lat = df_litowl["lat"].mean()
center_lng = df_litowl["lng"].mean()

m = folium.Map(location=[center_lat, center_lng], zoom_start=11)

# # Add markers
for _, row in df_litowl.iterrows():
    popup_text = (
        f"<b>{row['locName']}</b><br>"
        f"Species matched: {row['comName']}"
    )
    
    folium.Marker(
        location=[row["lat"], row["lng"]],
        popup=popup_text
    ).add_to(m)

m

In [ ]:
df_all.count()

In [ ]:
df_all[["locId"]].drop_duplicates().count()

In [ ]:
url = "https://api.ebird.org/v2/ref/region/list/subnational2/ES-CT"
response = requests.get(url, headers=headers)

regions = response.json()
regions

[{'code': 'ES-CT-BR', 'name': 'Barcelona'},
 {'code': 'ES-CT-GN', 'name': 'Girona'},
 {'code': 'ES-CT-LD', 'name': 'Lleida'},
 {'code': 'ES-CT-TG', 'name': 'Tarragona'}]

In [ ]:
import requests
import pandas as pd

url = "https://api.ebird.org/v2/ref/region/list/subnational1/IS"

headers = {"X-eBirdApiToken": API_KEY}

response = requests.get(url, headers=headers)
response.raise_for_status()

regions = response.json()

df_regions = pd.DataFrame(regions)
print(df_regions)

 code               name
0  IS-7         Austurland
1  IS-1   Höfuðborgarsvæði
2  IS-6  Norðurland eystra
3  IS-5  Norðurland vestra
4  IS-8          Suðurland
5  IS-2           Suðurnes
6  IS-4         Vestfirðir
7  IS-3         Vesturland

In [ ]:
import requests
from pathlib import Path
from datetime import date, timedelta
import time

headers = {"X-eBirdApiToken": API_KEY}

regions = [
    {"code": "IS-1", "name": "Capital"},
    {"code": "IS-2", "name": "Southern_Peninsula"},
    {"code": "IS-3", "name": "West"},
    {"code": "IS-4", "name": "Westfjords"},
    {"code": "IS-5", "name": "Northwest"},
    {"code": "IS-6", "name": "Northeast"},
    {"code": "IS-7", "name": "East"},
    {"code": "IS-8", "name": "South"}
]

out_dir = Path("ebird_IS_historic")
out_dir.mkdir(exist_ok=True)

start_year = 2020
end_year = 2025

for region in regions:

    region_code = region["code"]
    region_name = region["name"]

    region_dir = out_dir / region_name
    region_dir.mkdir(exist_ok=True)

    base_url = f"https://api.ebird.org/v2/data/obs/{region_code}/historic"

    for year in range(start_year, end_year + 1):

        start_date = date(year, 3, 20)
        end_date = date(year, 3, 28)

        current_date = start_date

        while current_date <= end_date:

            y, m, d = current_date.year, current_date.month, current_date.day

            url = f"{base_url}/{y}/{m}/{d}"
            filename = region_dir / f"{region_code}_{y}_{m:02d}_{d:02d}.json"
            if not filename.exists():
                response = requests.get(url, headers=headers)
    
                print(region_code, y, m, d, response.status_code)
    
                if response.status_code == 200 and response.text.strip():
                    filename.write_bytes(response.content)
    
                time.sleep(2)  # polite delay

            current_date += timedelta(days=1)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import json
import pandas as pd
from pathlib import Path

dfs = []

files = list(Path("ebird_IS_historic").rglob("*.json"))
for file in files:
    data = json.loads(file.read_text())
    if data:
        df = pd.DataFrame(data)
        df["source_file"] = file.name
        dfs.append(df)

df_is_all = pd.concat(dfs, ignore_index=True)

In [ ]:
# pd.set_option('display.max_rows', None)

In [ ]:
df_is_all[['comName']].drop_duplicates()

In [ ]:
is_wishlist = [
    'Common Eider', 
    'White-winged Scoter', 
    'Long-tailed Duck', 
    'Iceland Gull', 
    'Ruddy Turnstone', 
    'Red-breasted Merganser', 
    'Common Redshank', 
    'Purple Sandpiper',
    'Common Loon',
    'Harlequin Duck', 
    'Black Guillemot', 
    'Glaucous Gull', 
    'Black-legged Kittiwake',
    'Merlin',
    'Rock Ptarmigan',
    'European Golden-Plover',
    'Red Knot',
    'Red-throated Loon',
    'Barnacle Goose',
    'Black-tailed Godwit',
    'Greater White-fronted Goose',
    'Short-eared Owl',
    'King Eider',
    'Northern Gannet',
    'Ruff',
    'American Wigeon',
    'Common Goldeneye',
    "Barrow's Goldeneye",
    'Razorbill',
    'White-tailed Eagle',
    'Surf Scoter',
    'Velvet Scoter',
    'Bar-tailed Godwit',
    'Dunlin',
    'Parasitic Jaeger',
    'Common Murre',
    'Manx Shearwater',
    'Brant',
    'Thick-billed Murre',
    'Razorbill',
    'Black Scoter',
    'Atlantic Puffin',
]

In [ ]:
df_is_all.loc[lambda df:df['comName'].isin(is_wishlist)].sort_values(['obsDt'])

In [ ]:
rank_df = (
    df_is_all
    .loc[lambda df: df['comName'].isin(is_wishlist)]
    .groupby(['locName', 'comName'])
    .count()[['sciName']]
    .reset_index()
)

rank_df['rank'] = rank_df.groupby('comName')['sciName'].rank(
    method='dense',
    ascending=False
)

In [ ]:
df_is_all = df_is_all.merge(
    rank_df[['locName', 'comName', 'rank']],
    on=['locName', 'comName'],
    how='left'
)

In [ ]:
df_wishlist = df_is_all.loc[lambda df: df['comName'].isin(is_wishlist)]

In [ ]:
df_is_all.loc[lambda df: df['comName'].isin(is_wishlist)].loc[
    lambda df: df['comName'] == "Barrow's Goldeneye"
].sort_values('obsDt')

In [ ]:
import folium

df_one_species = df_wishlist.loc[
    lambda df: df['comName'] == "Northern Gannet"
]

center_lat = df_one_species["lat"].mean()
center_lng = df_one_species["lng"].mean()

m = folium.Map(location=[center_lat, center_lng], zoom_start=11)

# color scale for ranks
def rank_color(rank):
    if rank == 1:
        return "green"
    elif rank == 2:
        return "blue"
    elif rank == 3:
        return "orange"
    else:
        return "red"

for _, row in df_one_species.iterrows():

    popup_text = (
        f"<b>{row['locName']}</b><br>"
        f"<b>Species:</b> {row['comName']}<br>"
        f"<b>Date:</b> {row['obsDt']}<br>"
        f"<b>Number:</b> {row['howMany']}<br>"
        f"<b>Rank:</b> {row['rank']}"
    )

    folium.Marker(
        location=[row["lat"], row["lng"]],
        popup=popup_text,
        icon=folium.Icon(color=rank_color(row["rank"]))
    ).add_to(m)

m


In [ ]:
import folium
import pandas as pd
from branca.colormap import LinearColormap

# ---- Fix SettingWithCopyWarning safely ----
df_wishlist = df_wishlist.copy()

# ---- Safe datetime conversion (no deprecated args) ----
df_wishlist.loc[:, "obsDt"] = pd.to_datetime(df_wishlist["obsDt"], errors="coerce")

# ---- Aggregate per location ----
def sort_records(g):
    g = g.sort_values(["comName", "obsDt"])
    return pd.Series({
        "species_list": g["comName"].tolist(),
        "dates": [d.strftime("%Y-%m-%d %H:%M") if pd.notnull(d) else "" for d in g["obsDt"]],
        "counts": g["howMany"].tolist(),
        "avg_rank": g["rank"].mean(),
        "n_species": g["comName"].nunique()
    })

agg = (
    df_wishlist
    .groupby(["locName","lat","lng"], group_keys=False)
    .apply(sort_records, include_groups=False)
    .reset_index()
)


# ---- Map center ----
center_lat = agg["lat"].mean()
center_lng = agg["lng"].mean()

m = folium.Map(location=[center_lat, center_lng], zoom_start=6)

# ---- Color scale for rank ----
colormap = LinearColormap(
    colors=["green","yellow","red"],
    vmin=agg["avg_rank"].min(),
    vmax=agg["avg_rank"].max()
)

# ---- Add markers ----
for _, row in agg.iterrows():

    # Build species list HTML
    species_html = ""
    for sp, dt, ct in zip(row["species_list"], row["dates"], row["counts"]):
        species_html += f"{sp} — {dt} — {ct}<br>"

    popup_html = f"""
    <b>{row['locName']}</b><br>
    <b>Species count:</b> {row['n_species']}<br>
    <b>Average rank:</b> {row['avg_rank']:.2f}<br><br>
    {species_html}
    """

    folium.CircleMarker(
        location=[row["lat"], row["lng"]],
        radius=5 + row["n_species"]*1.5,
        color=colormap(row["avg_rank"]),
        fill=True,
        fill_opacity=0.85,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# ---- Legend ----
colormap.caption = "Average Rank (Green = Best)"
colormap.add_to(m)

m


In [ ]:
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson

df = df_wishlist.copy()

df["obsDt"] = pd.to_datetime(df["obsDt"], errors="coerce", format="mixed")
df = df.dropna(subset=["obsDt"])

center = [df["lat"].mean(), df["lng"].mean()]
m = folium.Map(location=center, zoom_start=6)

# ------------------------
# BUILD ALL FEATURES
# ------------------------
features = []

for _, r in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [r["lng"], r["lat"]],
        },
        "properties": {
            "time": r["obsDt"].isoformat(),
            "species": r["comName"],
            "popup": (
                f"<b>{r['comName']}</b><br>"
                f"{r['locName']}<br>"
                f"{r['obsDt']}<br>"
                f"Count: {r['howMany']}"
            )
        }
    })

# ------------------------
# SINGLE TIMELINE
# ------------------------
TimestampedGeoJson(
    {"type":"FeatureCollection","features":features},
    period="P1D",
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    date_options="YYYY-MM-DD",
).add_to(m)

# ------------------------
# DROPDOWN FILTER JS
# ------------------------
species_list = sorted(df["comName"].unique())

dropdown = f"""
<select id="speciesFilter" style="position: fixed; top: 10px; left: 50px; z-index:9999;">
<option value="All">All species</option>
{''.join([f'<option value="{s}">{s}</option>' for s in species_list])}
</select>

<script>

setTimeout(function() {{

    var tdLayer = Object.values(map._layers).find(l => l._baseLayer);

    document.getElementById("speciesFilter").addEventListener("change", function() {{

        var chosen = this.value;

        var base = tdLayer._baseLayer._layers;

        Object.values(base).forEach(function(layer){{
            var sp = layer.feature.properties.species;

            if(chosen === "All" || sp === chosen){{
                layer.setStyle ? layer.setStyle({{opacity:1, fillOpacity:1}}) :
                                 layer.setOpacity(1);
            }} else {{
                layer.setStyle ? layer.setStyle({{opacity:0, fillOpacity:0}}) :
                                 layer.setOpacity(0);
            }}
        }});

    }});

}}, 1000);
</script>
"""


m.get_root().html.add_child(folium.Element(dropdown))

m


In [ ]:
df_wishlist 
